# Pré-traitement des données 'Google'

* requète sur `BigQuerry`:
        /* 26Go proceceed */

        #standardSQL

        SELECT
          root.publication_number,
          publication_date,
          root.title_localized,
          root.cpc,
          root.citation
        FROM
          `patents-public-data.patents.publications` root,
          UNNEST( cpc ) AS cpc
        WHERE 
         cpc.code like 'A45D29/02' and
         country_code = 'US'
        LIMIT
          1000

In [1]:
%load_ext autoreload
%autoreload 2

In [8]:
import config as conf

import tools as tl

In [9]:
import json
import pickle

In [10]:
import datetime as dt

In [13]:
# load the data
file_path = conf.DATADIR + conf.QUERYFILE

data_brut = []
with open(file_path) as f:
    for line in f:
        entry = json.loads(line)
        data_brut.append( entry )
        
print( 'nombre de résultats bruts: %i' % len(data_brut) )

nombre de résultats bruts: 408


## Créer un dictionnaire { patent_number : infos, ... }

In [19]:
data = {}
k = 0  # debug
for entry in data_brut:
   
    
    pubnum = entry['publication_number']
    patent_number = tl.format_patentnumber( pubnum )
    
    if patent_number in data: # supprime les doublons
        continue

    new_entry = {}
    
    new_entry['publication_number_raw'] = pubnum
    new_entry['patent_number'] = patent_number
    
    # date
    new_entry['year'] = int(  entry['publication_date'][0:4] )   
    new_entry['month'] = int( entry['publication_date'][4:6] )
    new_entry['day'] = int(   entry['publication_date'][6:]  )
    date = dt.date(new_entry['year'], new_entry['month'], new_entry['day'])
    
    new_entry['date_str'] = entry['publication_date']
    new_entry['date_formatted'] = date.strftime( '%B %d, %Y' )
    
    # info
    new_entry['title'] = entry['title_localized'][0]['text'].strip('.')
    
    # abstract
    if len( entry['abstract_localized'] )>0:
        new_entry['abstract'] = entry['abstract_localized'][0]['text']
    else:
        new_entry['abstract'] = ''
        
    # description text
    if len( entry['description_localized'] )>0:
        new_entry['description'] = entry['description_localized'][0]['text']
    else:
        new_entry['description'] = ''
    
    # inventor & assignee
    new_entry['inventor'] = entry['inventor_harmonized']
    new_entry['assignee'] = entry['assignee_harmonized']
    
    # citations :
    citation = [] 
    for cit in entry['citation']:
        if not cit['publication_number'] : continue
        infos = {}
        infos['publication_number_raw'] = cit['publication_number'] 
        infos['patent_number'] = tl.format_patentnumber( cit['publication_number']  )

        infos['category'] =  cit['category'] 

        citation.append( infos )
        
    new_entry['citation'] = citation

    # CPC
    new_entry['cpc'] = entry['cpc']
    
    # save
    data[ patent_number ] = new_entry
    
    k+=1  # debug
    
print( 'nombre de brevets uniques: %i' % len(data) )

nombre de brevets uniques: 379


## Mise en forme des noms

In [20]:
# pour les inventeurs et les 'assignees':
for e in data.values():
    for i in e['inventor']:
        i['name_formatted'] = tl.format_name( i['name'] )
        
    for i in e['assignee']:
        i['name_formatted'] = tl.format_name( i['name'] )

## Ajoute les entrées : 'cited' and 'citedby'

In [27]:
# init
ghost = {}

for entry in data.values():
    entry['cited'] = []
    entry['citedby'] = []
    
# loop
for patent_number, entry in data.items():
    
    cited_number_list = [ c['patent_number'] for c in entry['citation']  ]
    
    for cited_number in cited_number_list:
        if cited_number in data:
            entry['cited'].append( cited_number )
            data[ cited_number ]['citedby'].append( patent_number )
        else:
            tl.countplusone( ghost, cited_number ) 
            
    
print( 'nombre de brevet cité mais non présent : %i' % len(ghost) )  # nombre des brevets cités mais non dans DATA

nombre de brevet cité mais non présent : 1086


In [29]:
# Statistique des brevets 'ghost'
sortedghost = sorted( ghost.items(), key= lambda x:x[1], reverse = True)
print( ' top10 : ' + ', '.join( ['%s (%i)' % nk for nk in sortedghost[:10] ] ) )

 top10 : US6220251 (17), US5546658 (16), US3744131 (16), US3838507 (15), US5392518 (14), US3903596 (14), US6523545 (14), US2955354 (13), US4856190 (12), US4196514 (12)


In [ ]:
pickle.dump( data, open('web/data/patent_infos.pickle','wb') )

## Cherche les images existantes  et retrouve les meta-données

In [34]:
print( '- figures -' )

- figures -


In [32]:
import os
import re

In [36]:
FIGURESLIST =  os.listdir( conf.FIGURESDIR )

print('nombre de figures :', len(FIGURESLIST))

nombre de figures : 3361


In [71]:
# init 
for entry in data.values():
    entry['figures'] = []

# loop
nopatentforfigure = {}
for k, figname in enumerate( FIGURESLIST ):

    patnum, fignum = tl.nums_from_figurename( figname  )

    # get figure size
    filepath = conf.FIGURESDIR + figname
    w, h = tl.get_imagesize( filepath )
       
    figinfo = {'filename':figname, 'number':fignum, 'width':w, 'height': h}

    # save
    if patnum in data:
        data[patnum]['figures'].append( figinfo )
    else:
        # pour le debug
        if patnum in nopatentforfigure:
            nopatentforfigure[patnum].append( figinfo )
        else:
            nopatentforfigure[patnum] = [ figinfo ]
         
    print( 'enregistre les images: %i/%i' % (k+1, len(FIGURESLIST)), end='\r' )
else:
    print('')
    
print('nombre de brevet hors DATA ayant une figure : ', len(nopatentforfigure))

enregistre les images: 624/3361

nombre de brevet hors DATA ayant une figure :  0


In [61]:
# Trie les figures par numéro - important -
# (à priori l'ordre d'apparition dans le brevet)
for entry in data.values():
    entry['figures'] = sorted(entry['figures'], key= lambda x: int( x['number'] ) )

In [60]:
# stats - Brevet sans figures :
patentwithoutfigure = [ patnum for patnum, e in data.items() if not e['figures'] ]
        
print( 'nombre de brevets sans figures : %i'% len(patentwithoutfigure))

print('  ' + ', '.join( patentwithoutfigure[:5] ) + (' ...' if len(patentwithoutfigure)>5 else '') )

nombre de brevets sans figures : 3
  US1135987, US20160029768, US20170231361


# Ajoute la legende

In [63]:
from extract_legend.extract_legend import extract_legend

In [67]:
# Loop
for i, e in  enumerate( data.values() ):
    legend, raw_legend = extract_legend( e['description'] )
    if legend:
        tl.add_consecutive_field( legend )  # to add space in the template
    e['legend'] = legend
    e['raw_legend'] = raw_legend
    
    print( 'extraie la légende: %i/%i' % (i+1, len(data)), end='\r' )
else:
    print('')

extraie la légende: 379/379


# Save

In [75]:
# save
pickle.dump( data, open(conf.DATADIR + conf.PATENTSINFOFILE,'wb') )

print( '%s  saved' % (conf.DATADIR + conf.PATENTSINFOFILE) )

data_nailclipper/patent_infos.pickle  saved


## Group by year